## install pip wheels

In [ ]:
#!conda install '../input/conda-packages/python-3.8.10-h49503c6_1_cpython.tar.bz2' -c conda-forge -y -q
!conda install '../input/conda-packages/libjpeg-turbo-2.1.0-h7f98852_0.tar.bz2' -c conda-forge -y -q
!conda install '../input/conda-packages/libgcc-ng-9.3.0-h2828fa1_19.tar.bz2' -c conda-forge -y -q
!conda install '../input/conda-packages/gdcm-2.8.9-py37h500ead1_1.tar.bz2' -c conda-forge -y -q
!conda install '../input/conda-packages/conda-4.10.1-py37h89c1867_0.tar.bz2' -c conda-forge -y -q
!conda install '../input/conda-packages/certifi-2020.12.5-py37h89c1867_1.tar.bz2' -c conda-forge -y -q
!conda install '../input/conda-packages/openssl-1.1.1k-h7f98852_0.tar.bz2' -c conda-forge -y -q

In [ ]:
!python --version

In [ ]:
#install pips
# !pip install ../input/pip-wheels/pylibjpeg_openjpeg-1.1.1-cp37-cp37m-manylinux2010_x86_64.whl --no-index
# !pip install ../input/pip-wheels/pylibjpeg_rle-1.1.0-cp37-cp37m-manylinux2010_x86_64.whl --no-index
# !pip install ../input/pip-wheels/pylibjpeg-1.3.0-py3-none-any.whl --no-index

!pip install ../input/pip-wheels/torch-1.9.0-cp37-cp37m-manylinux1_x86_64.whl --no-index --no-deps
!pip install ../input/pip-wheels/torchvision-0.10.0-cp37-cp37m-manylinux1_x86_64.whl --no-index --no-deps

!pip install ../input/pip-wheels/pycocotools-2.0.2.tar.gz --no-index
!pip install ../input/pip-wheels/albumentations-1.0.0-py3-none-any.whl --no-index
!pip install ../input/pip-wheels/pycocotools-2.0.2.tar.gz --no-index
!pip install ../input/pip-wheels/antlr4-python3-runtime-4.8.tar.gz --no-index
!pip install ../input/pip-wheels/omegaconf-2.1.0-py3-none-any.whl --no-index
!pip install ../input/pip-wheels/timm-0.4.13.zip --no-index
!pip install ../input/pip-wheels/effdet-0.2.4-py3-none-any.whl --no-index 

!pip install ../input/pip-wheels/pretrainedmodels-0.7.4.tar.gz --no-index --no-deps
!pip install ../input/pip-wheels/efficientnet_pytorch-0.6.3.tar.gz --no-index --no-deps
!pip install ../input/pip-wheels/segmentation_models_pytorch-0.2.0-py3-none-any.whl --no-index --no-deps

!pip install ../input/pip-wheels/ensemble_boxes-1.0.6-py3-none-any.whl --no-index

In [ ]:
!mkdir configs
!mkdir data
!mkdir models

!cp ../input/kaggle-siim-covid-detection-9th-place/* ./
!tar -xvf ./configs.tar -C ./configs >> /dev/null
!tar -xvf ./data.tar -C ./data >> /dev/null
!tar -xvf ./models.tar -C ./models >> /dev/null
!rm ./*.tar

In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy as sp
import os
import timm
import sys
import os
import importlib
import multiprocessing as mp
import cv2
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import glob
import torch
from copy import copy
from torch.cuda.amp import GradScaler, autocast

from torch.utils.data import DataLoader

torch.backends.cudnn.benchmark = True
cv2.setNumThreads(0)

In [ ]:
sys.path.append('./configs')
sys.path.append('./data')
sys.path.append('./models')

In [ ]:
COMP_FOLDER = '../input/siim-covid19-detection/'


SAMPLE_SUBMISSION = pd.read_csv(COMP_FOLDER + 'sample_submission.csv').set_index('id')
N_CORES = mp.cpu_count()

RAM_CHECK = False
MIXED_PRECISION = False
DEVICE = "cuda"

if len(SAMPLE_SUBMISSION) == 2477:
    DEBUG = True
else:
    DEBUG = False

In [ ]:
# convert all dicoms to png

from pydicom import read_file
from pydicom.pixel_data_handlers.util import apply_voi_lut

def read_xray(path, voi_lut = True, fix_monochrome = True):
    # Original from: https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way
    dicom = read_file(path)
    
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to 
    # "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
        
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
        
    return data

def do_one(fp):
    img = read_xray(fp)
    h, w = img.shape
    
    study_id, series_id, image_id = fp.split('/')[4:]
    
    img_025 = cv2.resize(img, (0, 0), fx=0.25, fy = 0.25)
    folder = f'test_025/{study_id}/{series_id}/'
    if not os.path.exists(folder):
        os.makedirs(folder)
    cv2.imwrite(folder + image_id.replace('dcm','png'),img_025)
    
    img_05 = cv2.resize(img, (0, 0), fx=0.5, fy = 0.5)
    folder = f'test_05/{study_id}/{series_id}/'
    if not os.path.exists(folder):
        os.makedirs(folder)
    cv2.imwrite(folder + image_id.replace('dcm','png'),img_05)
        
    return image_id, study_id, series_id, w, h

In [ ]:
fps = []
for dirname, _, filenames in os.walk(f'{COMP_FOLDER}test'):
    for file in filenames:
        fps += [dirname + '/' + file]
        
if DEBUG:
    fps = fps[:50]

In [ ]:
with mp.Pool(N_CORES) as p:
    res = list(tqdm(p.imap(do_one, fps),total=len(fps)))
test_info = np.array(res)

In [ ]:
test_df = pd.DataFrame({'PredictionString':['none 1 0 0 1 1']* len(test_info)})
test_df[['id','StudyInstanceUID','series_id','width','height']] = test_info
test_df['id'] = test_df['id'].str.replace('.dcm','_image')
test_df[['width','height']] = test_df[['width','height']].astype(int)
test_df.to_csv('test_image_level_v2.csv',index=False)
test_df.head()

In [ ]:
#helper functions

def get_state_dict(sd_fp):
    sd = torch.load(sd_fp, map_location="cpu")['model']
    #fix DDP if necessary
    sd = {k.replace("module.", ""):v for k,v in sd.items()}
    return sd

def pp_transform(logits, pp_transformation):
    if pp_transformation == "softmax":
        pred = logits.softmax(1)
    elif pp_transformation == "sigmoid":
        pred = logits.sigmoid()
    else:
        pred = logits.detach()
    return pred

In [ ]:
def run_study(config_file, state_dicts):
    cfg = importlib.import_module('default_config')
    importlib.reload(cfg)
    cfg = importlib.import_module(config_file)
    importlib.reload(cfg)
    cfg = copy(cfg.cfg)
    print(cfg.model, cfg.dataset, cfg.backbone, cfg.pretrained_weights)

    cfg.test_df = 'test_image_level_v2.csv'
    cfg.test_data_folder = f"test_{str(cfg.scale).replace('.','')}/"
    print(config_file, cfg.test_data_folder)
    cfg.pretrained = False

    ds = importlib.import_module(cfg.dataset)
    importlib.reload(ds)

    CustomDataset = ds.CustomDataset
    batch_to_device = ds.batch_to_device

    cfg.batch_size = 8
    test_ds = CustomDataset(test_df, cfg, cfg.val_aug, mode="test")
    test_dl = DataLoader(test_ds, shuffle=False, batch_size = cfg.batch_size, num_workers = N_CORES, pin_memory=True)

    model = importlib.import_module(cfg.model)
    importlib.reload(model)
    Net = model.Net

    nets = []

    for i,state_dict in enumerate(state_dicts):
        net = Net(cfg).eval().cuda()
        sd = get_state_dict(state_dict)
        print("loading dict")
        net.load_state_dict(sd, strict=True)
        nets += [net]

    #with torch.no_grad():    
    with torch.inference_mode():

        preds_0 = []
        for batch in tqdm(test_dl):
            batch = batch_to_device(batch, DEVICE)
            preds_ = []
            for net in nets:
                with autocast():
                    logits = net(batch)['class_logits'].float()
                #pred = pp_transform(logits, cfg.pp_transformation)
                pred = logits
                preds_ += [pred.cpu().numpy()]

            preds_0 += [np.array(preds_)]
        preds_0 = np.concatenate(preds_0, axis=1)
    preds_0.shape
    return preds_0

preds = []

state_dicts = []
for fold, seed in enumerate([487735,464165,633237,686759,203682]):
    state_dicts.append(f'../input/weights-cfg-ch-study-xcit-2e15/fold{fold}/checkpoint_last_seed{seed}.pth')
print(state_dicts)

p = run_study("cfg_ch_study_xcit_2e15", state_dicts)
preds.append(p)

state_dicts = []
for fold, seed in enumerate([226896,259854,476990,577158,258586]):
    state_dicts.append(f'../input/weights-cfg-ch-study-xcit-2d/fold{fold}/checkpoint_last_seed{seed}.pth')
print(state_dicts)

p = run_study("cfg_ch_study_xcit_2d", state_dicts)
preds.append(p)

state_dicts = []
for file in glob.glob("../input/cfg-ps-study-ben-7-weights/*.pth"):
    state_dicts.append(file)
print(state_dicts)

p = run_study("cfg_ps_study_ben_7", state_dicts)
preds.append(p)

state_dicts = []
for file in glob.glob("../input/cfg-ps-study-7-weights/*.pth"):
    state_dicts.append(file)
print(state_dicts)

p = run_study("cfg_ps_study_7", state_dicts)
preds.append(p)

state_dicts = []
for fold, seed in enumerate([322511,301949,777691,615690,207310]):
    state_dicts.append(f'../input/weights-cfg-ch-study-12d/fold{fold}/checkpoint_last_seed{seed}.pth')
print(state_dicts)

p = run_study("cfg_ch_study_12d", state_dicts)
preds.append(p)

state_dicts = []
for fold, seed in enumerate([825690,473862,975082,699448,879221]):
    state_dicts.append(f'../input/weights-cfg-ch-study-12d/fold{fold}/checkpoint_last_seed{seed}.pth')
print(state_dicts)

p = run_study("cfg_ch_study_12d", state_dicts)
preds.append(p)

state_dicts = []
for fold in range(5):
    for file in glob.glob(f"../input/weights-cfg-ch-study-7-1024b5/fold{fold}/*.pth"):
        state_dicts.append(file)
    #state_dicts.append(f'../input/weights-cfg-ch-study-12d/fold{fold}/checkpoint_last_seed{seed}.pth')
print(state_dicts)
print(len(state_dicts))


p = run_study("cfg_ch_study_7_1024b5", state_dicts)
preds.append(p)

state_dicts = []
for fold, seed in enumerate([775009,776562,239357,624529,909642]):
    state_dicts.append(f'../input/weights-cfg-ch-study-7-1024b7-2/fold{fold}/checkpoint_last_seed{seed}.pth')
print(state_dicts)

p = run_study("cfg_ch_study_7_1024b7_2", state_dicts)
preds.append(p)

#preds = [sp.special.softmax(p, axis=2).mean(axis=0) for p in preds]
preds = np.concatenate(preds, axis=0)
preds = sp.special.softmax(preds, axis=2)
preds = np.mean(preds, axis=0)

print(preds.shape)

study_ids = test_df['StudyInstanceUID']

In [ ]:
def create_pred_str_study(row):
    items = []
    for label in ["negative", "typical", "indeterminate", "atypical"]:
        items += [label,row[label],'0','0','1','1']
    return ' '.join([str(i) for i in items])

def create_sub(test_preds):
    df_study = pd.DataFrame({'id':study_ids})
    df_study['id'] = df_study['id'].astype(str) + '_study'
    df_study[["negative", "typical", "indeterminate", "atypical"]] = test_preds
    df_study = df_study.groupby('id').agg('mean').reset_index()
    df_study_orig = df_study.copy()
    df_study['PredictionString'] = df_study.apply(lambda row: create_pred_str_study(row), axis = 1)
    df_study= df_study[['id','PredictionString']].copy()
    df_study_orig["study_id"] = df_study_orig["id"].str.replace("_study", "")
    return df_study, df_study_orig

test_preds_study = preds
test_preds_study.shape

df_study, df_study_orig = create_sub(test_preds_study)
df_study = df_study.set_index('id')
df_study.head()

SAMPLE_SUBMISSION.loc[df_study.index,'PredictionString'] = df_study['PredictionString'].values

In [ ]:
TOPK_EFFDET = 75

def get_df_image_pred_effdet(cfg, detections, image_ids):

    images = []
    boxes = []
    labels = []
    confs = []
    
    for i,det in enumerate(detections):

        detection = det#[det[:,4] > cfg.box_min_conf]
        
        images.append([image_ids[i]] * len(detection))
        labels.append(['opacity'] * len(detection))
        boxes.append((detection[:,:4] / cfg.image_size[0]).clip(0,1))
        confs.append(detection[:,4])
        images += [[image_ids[i]]]
        labels += [['none']]
        boxes += [np.array([0,0,1,1])[None,:]]
        confs += [np.array([1-det[:,4].max()])]
    
    confs = np.concatenate(confs)
    labels = np.concatenate(labels)
    images = np.concatenate(images)
    boxes = np.concatenate(boxes)
    df_image_pred = pd.DataFrame({'ImageID':images,'LabelName':labels,'Conf':confs})
    df_image_pred[['XMin','YMin','XMax','YMax']] = boxes

    df_image_pred = df_image_pred[df_image_pred.XMin != df_image_pred.XMax]
    df_image_pred = df_image_pred[df_image_pred.YMin != df_image_pred.YMax]

    df_image_pred = df_image_pred[['ImageID','LabelName','Conf','XMin', 'XMax', 'YMin', 'YMax']].copy()
    
    return df_image_pred

def get_results_effdet(config_file, state_dicts):
    cfg = importlib.import_module('default_config')
    importlib.reload(cfg)
    cfg = importlib.import_module(config_file)
    importlib.reload(cfg)
    cfg = copy(cfg.cfg)
    print(cfg.model, cfg.dataset, cfg.backbone, cfg.pretrained_weights)

    cfg.test_df = 'test_image_level_v2.csv'
    cfg.test_data_folder = f"test_{str(cfg.scale).replace('.','')}/"
    print(config_file, cfg.test_data_folder)
    cfg.pretrained = False
    cfg.pretrained_convhead = False

    ds = importlib.import_module(cfg.dataset)
    importlib.reload(ds)

    CustomDataset = ds.CustomDataset
    batch_to_device = ds.batch_to_device

    cfg.batch_size = 8
    test_ds = CustomDataset(test_df, cfg, cfg.val_aug, mode="test")
    test_dl = DataLoader(test_ds, shuffle=False, batch_size = cfg.batch_size, num_workers = N_CORES, pin_memory=True)

    model = importlib.import_module(cfg.model)
    importlib.reload(model)
    Net = model.Net

    nets = []

    for i,state_dict in enumerate(state_dicts):
        net = Net(cfg).eval().cuda()
        sd = get_state_dict(state_dict)
        print("loading dict")
        net.load_state_dict(sd, strict=True)
        nets += [net]

    #with torch.no_grad():    
    with torch.inference_mode():

        preds_0 = []
        for batch in tqdm(test_dl):
            batch = batch_to_device(batch, DEVICE)
            preds_ = []
            for net in nets:
                with autocast():
                    detections_ = net(batch)['detections'].float()
                    detections_ = detections_[:,:TOPK_EFFDET,:]
                #pred = pp_transform(logits, cfg.pp_transformation)
                preds_ += [detections_.cpu().numpy()]
                
            preds_0 += [np.array(preds_)]
        preds_0 = np.concatenate(preds_0, axis=1)
        
        
    dfs_pred = []
    for fold in tqdm([0,1,2,3,4]):
        df_image_pred = get_df_image_pred_effdet(cfg, preds_0[fold], test_df['id'])
        dfs_pred += [df_image_pred]
    dfs_pred = pd.concat(dfs_pred)
    
    
    
    return dfs_pred

In [ ]:
config_file = 'cfg_ps_35'
state_dicts = []
for file in glob.glob("../input/cfg-ps-35-weights/*.pth"):
    state_dicts.append(file)
print(state_dicts)

dfs_pred_ps_35 = get_results_effdet(config_file, state_dicts)
dfs_pred_ps_35.sort_values(['ImageID','Conf'])

dfs_pred_ps_35.head()

In [ ]:
config_file = 'cfg_ps_27'
state_dicts = []
for file in glob.glob("../input/cfg-ps-27-weights/*.pth"):
    state_dicts.append(file)
print(state_dicts)

In [ ]:
dfs_pred_ps_27 = get_results_effdet(config_file, state_dicts)
dfs_pred_ps_27.sort_values(['ImageID','Conf'])

In [ ]:
dfs_pred_ps_27.head()

In [ ]:
config_file = 'cfg_ps_13'
state_dicts = []
for file in glob.glob("../input/cfg-ps-13-weights/*.pth"):
    state_dicts.append(file)
print(state_dicts)

In [ ]:
dfs_pred_ps_13 = get_results_effdet(config_file, state_dicts)

In [ ]:
dfs_pred_ps_13.sort_values(['ImageID','Conf'])

In [ ]:
config_file = 'cfg_ps_9'
state_dicts = []
for file in glob.glob("../input/cfg-ps-9-weights/*.pth"):
    state_dicts.append(file)
print(state_dicts)

In [ ]:
dfs_pred_ps_9 = get_results_effdet(config_file, state_dicts)

In [ ]:
dfs_pred_ps_9.sort_values(['ImageID','Conf'])

In [ ]:
import yaml

run1 = dict(
    path = '',
    train = '/kaggle/working/test_05',
    val = '/kaggle/working/test_05',
    test = '/kaggle/working/test_05',
    nc =  1,  # number of classes
    names = [ '0. opacity' ],  # class names

)

run2 = dict(
    path = '',
    train = '/kaggle/working/test_05',
    val = '/kaggle/working/test_05',
    test = '/kaggle/working/test_05',
    nc =  2,  # number of classes
    names = [ '0. opacity', '1. none' ],  # class names

)


with open('data_1cls.yaml', 'w') as outfile:
    yaml.dump(run1, outfile, default_flow_style=False)
    
with open('data_2cls.yaml', 'w') as outfile:
    yaml.dump(run2, outfile, default_flow_style=False)

In [ ]:
# 

In [ ]:
!python ../input/yolov5-github/yolov5/test.py --conf-thres 0.0001 --weights ../input/weights-yolo/run_ch_1b/fold0/weights/last.pt --data /kaggle/working/data_1cls.yaml --augment --imgsz 512 --task test --save-txt --save-conf --project "" --name run_ch_1b/fold0/test_tta >>yolo_run_ch_1b.logs
!python ../input/yolov5-github/yolov5/test.py --conf-thres 0.0001 --weights ../input/weights-yolo/run_ch_1b/fold1/weights/last.pt --data /kaggle/working/data_1cls.yaml --augment --imgsz 512 --task test --save-txt --save-conf --project "" --name run_ch_1b/fold1/test_tta >>yolo_run_ch_1b.logs
!python ../input/yolov5-github/yolov5/test.py --conf-thres 0.0001 --weights ../input/weights-yolo/run_ch_1b/fold2/weights/last.pt --data /kaggle/working/data_1cls.yaml --augment --imgsz 512 --task test --save-txt --save-conf --project "" --name run_ch_1b/fold2/test_tta >>yolo_run_ch_1b.logs
!python ../input/yolov5-github/yolov5/test.py --conf-thres 0.0001 --weights ../input/weights-yolo/run_ch_1b/fold3/weights/last.pt --data /kaggle/working/data_1cls.yaml --augment --imgsz 512 --task test --save-txt --save-conf --project "" --name run_ch_1b/fold3/test_tta >>yolo_run_ch_1b.logs
!python ../input/yolov5-github/yolov5/test.py --conf-thres 0.0001 --weights ../input/weights-yolo/run_ch_1b/fold4/weights/last.pt --data /kaggle/working/data_1cls.yaml --augment --imgsz 512 --task test --save-txt --save-conf --project "" --name run_ch_1b/fold4/test_tta >>yolo_run_ch_1b.logs

In [ ]:
!python ../input/yolov5-github/yolov5/test.py --conf-thres 0.0001 --weights ../input/weights-yolo/run_ch_2b/fold0/weights/last.pt --data /kaggle/working/data_1cls.yaml --augment --imgsz 512 --task test --save-txt --save-conf --project "" --name run_ch_2b/fold0/test_tta >>yolo_run_ch_2b.logs
!python ../input/yolov5-github/yolov5/test.py --conf-thres 0.0001 --weights ../input/weights-yolo/run_ch_2b/fold1/weights/last.pt --data /kaggle/working/data_1cls.yaml --augment --imgsz 512 --task test --save-txt --save-conf --project "" --name run_ch_2b/fold1/test_tta >>yolo_run_ch_2b.logs
!python ../input/yolov5-github/yolov5/test.py --conf-thres 0.0001 --weights ../input/weights-yolo/run_ch_2b/fold2/weights/last.pt --data /kaggle/working/data_1cls.yaml --augment --imgsz 512 --task test --save-txt --save-conf --project "" --name run_ch_2b/fold2/test_tta >>yolo_run_ch_2b.logs
!python ../input/yolov5-github/yolov5/test.py --conf-thres 0.0001 --weights ../input/weights-yolo/run_ch_2b/fold3/weights/last.pt --data /kaggle/working/data_1cls.yaml --augment --imgsz 512 --task test --save-txt --save-conf --project "" --name run_ch_2b/fold3/test_tta >>yolo_run_ch_2b.logs
!python ../input/yolov5-github/yolov5/test.py --conf-thres 0.0001 --weights ../input/weights-yolo/run_ch_2b/fold4/weights/last.pt --data /kaggle/working/data_1cls.yaml --augment --imgsz 512 --task test --save-txt --save-conf --project "" --name run_ch_2b/fold4/test_tta >>yolo_run_ch_2b.logs

In [ ]:
!python ../input/yolov5-github/yolov5/test.py --conf-thres 0.0001 --weights ../input/weights-yolo/run_ch_3b/fold0/weights/last.pt --data /kaggle/working/data_1cls.yaml --augment --imgsz 640 --task test --save-txt --save-conf --project "" --name run_ch_3b/fold0/test_tta >>yolo_run_ch_3b.logs
!python ../input/yolov5-github/yolov5/test.py --conf-thres 0.0001 --weights ../input/weights-yolo/run_ch_3b/fold1/weights/last.pt --data /kaggle/working/data_1cls.yaml --augment --imgsz 640 --task test --save-txt --save-conf --project "" --name run_ch_3b/fold1/test_tta >>yolo_run_ch_3b.logs
!python ../input/yolov5-github/yolov5/test.py --conf-thres 0.0001 --weights ../input/weights-yolo/run_ch_3b/fold2/weights/last.pt --data /kaggle/working/data_1cls.yaml --augment --imgsz 640 --task test --save-txt --save-conf --project "" --name run_ch_3b/fold2/test_tta >>yolo_run_ch_3b.logs
!python ../input/yolov5-github/yolov5/test.py --conf-thres 0.0001 --weights ../input/weights-yolo/run_ch_3b/fold3/weights/last.pt --data /kaggle/working/data_1cls.yaml --augment --imgsz 640 --task test --save-txt --save-conf --project "" --name run_ch_3b/fold3/test_tta >>yolo_run_ch_3b.logs
!python ../input/yolov5-github/yolov5/test.py --conf-thres 0.0001 --weights ../input/weights-yolo/run_ch_3b/fold4/weights/last.pt --data /kaggle/working/data_1cls.yaml --augment --imgsz 640 --task test --save-txt --save-conf --project "" --name run_ch_3b/fold4/test_tta >>yolo_run_ch_3b.logs

In [ ]:
!python ../input/yolov5-github/yolov5/test.py --conf-thres 0.0001 --weights ../input/weights-yolo/run_ch_7_2cls_2/fold0/weights/last.pt --data /kaggle/working/data_2cls.yaml --augment --imgsz 512 --task test --save-txt --save-conf --project "" --name run_ch_7_2cls_2/fold0/test_tta >>yolo_run_ch_7_2cls_2.logs
!python ../input/yolov5-github/yolov5/test.py --conf-thres 0.0001 --weights ../input/weights-yolo/run_ch_7_2cls_2/fold1/weights/last.pt --data /kaggle/working/data_2cls.yaml --augment --imgsz 512 --task test --save-txt --save-conf --project "" --name run_ch_7_2cls_2/fold1/test_tta >>yolo_run_ch_7_2cls_2.logs
!python ../input/yolov5-github/yolov5/test.py --conf-thres 0.0001 --weights ../input/weights-yolo/run_ch_7_2cls_2/fold2/weights/last.pt --data /kaggle/working/data_2cls.yaml --augment --imgsz 512 --task test --save-txt --save-conf --project "" --name run_ch_7_2cls_2/fold2/test_tta >>yolo_run_ch_7_2cls_2.logs
!python ../input/yolov5-github/yolov5/test.py --conf-thres 0.0001 --weights ../input/weights-yolo/run_ch_7_2cls_2/fold3/weights/last.pt --data /kaggle/working/data_2cls.yaml --augment --imgsz 512 --task test --save-txt --save-conf --project "" --name run_ch_7_2cls_2/fold3/test_tta >>yolo_run_ch_7_2cls_2.logs
!python ../input/yolov5-github/yolov5/test.py --conf-thres 0.0001 --weights ../input/weights-yolo/run_ch_7_2cls_2/fold4/weights/last.pt --data /kaggle/working/data_2cls.yaml --augment --imgsz 512 --task test --save-txt --save-conf --project "" --name run_ch_7_2cls_2/fold4/test_tta >>yolo_run_ch_7_2cls_2.logs

In [ ]:
TOPK = 75

def xywh2x1x2y1y2(x):
    # Convert nx4 boxes from [x, y, w, h] to [x1, y1, x2, y2] where xy1=top-left, xy2=bottom-right
    y = x.clone() if isinstance(x, torch.Tensor) else np.copy(x)
    y[:, 0] = x[:, 0] - x[:, 2] / 2  # top left x
    y[:, 1] = x[:, 0] + x[:, 2] / 2  # bottom right x
    y[:, 2] = x[:, 1] - x[:, 3] / 2  # top left y
    y[:, 3] = x[:, 1] + x[:, 3] / 2  # bottom right y
    return y

LAB = np.array(['opacity','none'])

def get_results_yolo(val_df, pred_folder):


    results = {}

    images = []
    boxes = []
    labels = []
    targets = []
    confs = []
    target_boxes = []
    study_ids = []
    image_ids = val_df['id'].values
    detections = []
    missing_preds = []
    n_boxes = 0
    for i,id_ in enumerate(image_ids):

        w = val_df.iloc[i]['width']
        h = val_df.iloc[i]['height']
        if 'label' in val_df.columns:
            label_box = np.array(val_df.iloc[i]['label'].split()).reshape(-1,6)
            box_target = label_box[:,2:].astype(np.float32)
            target = label_box[:,0]
            mask = target == 'opacity'
            if mask.sum() > 0:
                box_target[mask,0] /= w
                box_target[mask,2] /= w
                box_target[mask,1] /= h
                box_target[mask,3] /= h
            target_boxes.append(box_target)
            targets.append(target)
        try:
            det = pd.read_csv(f'{pred_folder}labels/{id_.replace("_image","")}.txt', sep=' ', header=None, names=['cls','x','y','w','h','Conf'])
            det = det.values
            det = det[det[:,0]==0]
            det = det[det[:, 5].argsort()[::-1]]
            det = det[:TOPK]
        except:
            det = np.empty((1,6))
            missing_preds += [id_]

        n_boxes += len(det)
        images.append([image_ids[i]] * len(det))
        labels.append(list(LAB[det[:,0].astype(int)]))
        box = xywh2x1x2y1y2(det[:,1:-1])
        box = box.clip(0,1)
#         box[:,:2] *= w
#         box[:,2:] *= h
        boxes.append(box)
        confs.append(det[:,5])

        study_ids.append(val_df.iloc[i]['StudyInstanceUID'])
#     print(len(no_preds))
    #     images += [image_ids[i]]
    #     labels += ['none']
    #     boxes += [np.array([0,0,1,1])[None,:]]
    #     confs += [1-det[:,5].max()]

    results["images"] = images
    results["boxes"] = boxes
    results["box_target"] = target_boxes
    results["target"] = targets
    results["labels"] = labels
    results["confs"] = confs
    results["study_idx"] = study_ids
    print(n_boxes)
    return results, missing_preds

def get_df_image_pred(results, single_class =True):
    images = []
    boxes = []
    labels = []
    confs = []


    for i in range(len(results["boxes"])):

        images += results["images"][i]
        labels +=results["labels"][i]
        boxes += [results["boxes"][i]]
        confs += list(results["confs"][i])
        images += [results["images"][i][0]]
        labels += ['none']
        boxes += [np.array([0,1,0,1])[None,:]]
        confs += [1-results["confs"][i].max()]


    df_image_pred = pd.DataFrame({'ImageID':images,'LabelName':labels,'Conf':np.array(confs)})
    df_image_pred[['XMin','XMax','YMin','YMax']] = np.concatenate(boxes)

    df_image_pred = df_image_pred[df_image_pred.XMin != df_image_pred.XMax]
    df_image_pred = df_image_pred[df_image_pred.YMin != df_image_pred.YMax]

    df_image_pred = df_image_pred[['ImageID','LabelName','Conf','XMin', 'XMax', 'YMin', 'YMax']].copy()
    return df_image_pred



In [ ]:
dfs_pred_1b = []
missing_preds = 0
for fold in tqdm([0,1,2,3,4]):
    pred_folder = f'run_ch_1b/fold{fold}/test_tta/'
#     print(len(os.listdir(pred_folder + 'labels/')))
    results, m = get_results_yolo(test_df, pred_folder)
    missing_preds += len(m)
    df_image_pred = get_df_image_pred(results)
    dfs_pred_1b += [df_image_pred]
dfs_pred_1b = pd.concat(dfs_pred_1b)
missing_preds

In [ ]:
dfs_pred_1b.sort_values(['ImageID','Conf'])

In [ ]:
dfs_pred_2b = []
for fold in tqdm([0,1,2,3,4]):
    pred_folder = f'run_ch_2b/fold{fold}/test_tta/'
#     print(len(os.listdir(pred_folder + 'labels/')))
    results, m = get_results_yolo(test_df, pred_folder)
    missing_preds += len(m)
    df_image_pred = get_df_image_pred(results)
    dfs_pred_2b += [df_image_pred]
dfs_pred_2b = pd.concat(dfs_pred_2b)
missing_preds

In [ ]:
dfs_pred_3b = []
for fold in tqdm([0,1,2,3,4]):
    pred_folder = f'run_ch_3b/fold{fold}/test_tta/'
#     print(len(os.listdir(pred_folder + 'labels/')))
    results, m = get_results_yolo(test_df, pred_folder)
    missing_preds += len(m)
    df_image_pred = get_df_image_pred(results)
    dfs_pred_3b += [df_image_pred]
dfs_pred_3b = pd.concat(dfs_pred_3b)
missing_preds

In [ ]:
dfs_pred_7_2cls_2 = []
for fold in tqdm([0,1,2,3,4]):
    pred_folder = f'run_ch_7_2cls_2/fold{fold}/test_tta/'
#     print(len(os.listdir(pred_folder + 'labels/')))
    results, m = get_results_yolo(test_df, pred_folder)
    missing_preds += len(m)
    df_image_pred = get_df_image_pred(results,single_class=False)
    dfs_pred_7_2cls_2 += [df_image_pred]
dfs_pred_7_2cls_2 = pd.concat(dfs_pred_7_2cls_2)
missing_preds

In [ ]:
df_image_pred_test = [dfs_pred_1b,dfs_pred_2b,dfs_pred_3b,dfs_pred_7_2cls_2,dfs_pred_ps_9,dfs_pred_ps_13,dfs_pred_ps_27,dfs_pred_ps_35]

In [ ]:
df_test_opacity = [df[df['LabelName'] == 'opacity'].copy() for df in df_image_pred_test]
df_test_none = [df[df['LabelName'] == 'none'].copy() for df in df_image_pred_test]

In [ ]:
df_test_opacity = pd.concat(df_test_opacity)

In [ ]:
df_test_opacity

In [ ]:
dfs_test_none = []
for item in df_test_none:
    df_image_pred_none = item[['ImageID','Conf','XMin','XMax','YMin','YMax']].groupby('ImageID').agg('max')
    df_image_pred_none = df_image_pred_none.reset_index()
    df_image_pred_none['LabelName'] = 'none'
    df_image_pred_none[['XMin','YMin','XMax','YMax']] = [0,0,1,1]
    dfs_test_none += [df_image_pred_none]
dfs_test_none = pd.concat(dfs_test_none)

In [ ]:
dfs_test_none2 = dfs_test_none[['ImageID','Conf','XMin','XMax','YMin','YMax']].groupby('ImageID').agg('mean')
dfs_test_none2 = dfs_test_none2.reset_index()
dfs_test_none2['LabelName'] = 'none'
dfs_test_none2[['XMin','YMin','XMax','YMax']] = [0,0,1,1]
dfs_test_none2

In [ ]:
test_df

In [ ]:
dfs_test_none2 = dfs_test_none2.merge(test_df[["id", "StudyInstanceUID"]], left_on="ImageID", right_on="id")

In [ ]:
dfs_test_none2 = dfs_test_none2.merge(df_study_orig[["study_id", "negative", "atypical"]], left_on="StudyInstanceUID", right_on="study_id")

In [ ]:
# PP 3
dfs_test_none2["Conf"] = 0.5*dfs_test_none2["Conf"] + 0.5*dfs_test_none2["negative"] + 0.2*dfs_test_none2["atypical"]

In [ ]:
# def create_pred_str_study(row):
#     items = []
#     for label in ["negative", "typical", "indeterminate", "atypical"]:
#         items += [label,row[label],'0','0','1','1']
#     return ' '.join([str(i) for i in items])

# def create_sub(test_preds, none_preds_image):
#     df_study = pd.DataFrame({'id':study_ids})
#     df_study['id'] = df_study['id'].astype(str) + '_study'
#     df_study[["negative", "typical", "indeterminate", "atypical"]] = test_preds
#     df_study = df_study.groupby('id').agg('mean').reset_index()
#     df_study["study_id"] = df_study["id"].str.replace("_study", "")
#     df_study = df_study.merge(none_preds_image, on="study_id")
    
#     #PP 2
#     df_study["negative"] = df_study["Conf"]
#     df_study["id"] = df_study["id_x"]
    
#     df_study_orig = df_study.copy()
#     df_study['PredictionString'] = df_study.apply(lambda row: create_pred_str_study(row), axis = 1)
#     df_study= df_study[['id','PredictionString']].copy()
#     df_study_orig["study_id"] = df_study_orig["id"].str.replace("_study", "")
#     return df_study, df_study_orig
# df_study, df_study_orig = create_sub(test_preds_study, dfs_test_none2)
# df_study = df_study.set_index('id')
# df_study.head()

# SAMPLE_SUBMISSION.loc[df_study.index,'PredictionString'] = df_study['PredictionString'].values

In [ ]:
del dfs_test_none2["negative"]
del dfs_test_none2["atypical"]
del dfs_test_none2["StudyInstanceUID"]
del dfs_test_none2["study_id"]

In [ ]:
dfs_test_none2

In [ ]:
from ensemble_boxes import weighted_boxes_fusion, non_maximum_weighted
from joblib import Parallel, delayed
# import warnings
# import sys
# if not sys.warnoptions:
#     warnings.simplefilter("ignore")

#def run_blend_boxes():
def run_blend_boxes(df_image,image_id):
    label = df_image['LabelName'].map({'opacity':0,'none':1}).values
    box = df_image[['XMin','YMin','XMax','YMax']].values
    conf = df_image['Conf'].values
    box, conf, label  = weighted_boxes_fusion([box], [conf], [label], weights=None, iou_thr=0.5, conf_type="max", skip_box_thr=0.0)
    boxes = box
    confs = conf
    labels = label
    image_ids = [image_id]*len(conf)
    
    return boxes, confs, labels, image_ids

def combine_boxes(df_image_pred):

    image_ids_unique = df_image_pred['ImageID'].unique()
    df_image_pred_gr = df_image_pred.groupby('ImageID')
    boxes = []
    confs = []
    labels = []
    image_ids = []
    
    res = Parallel(n_jobs=N_CORES)(delayed(run_blend_boxes)(df_image_pred_gr.get_group(image_id), image_id) for image_id in tqdm(image_ids_unique))

    boxes = [x[0] for x in res]
    confs = [x[1] for x in res]
    labels = [x[2] for x in res]
    image_ids = [x[3] for x in res]
    
    boxes = np.concatenate(boxes)
    confs = np.concatenate(confs)
    labels = np.concatenate(labels)
    image_ids = np.concatenate(image_ids)

    df = pd.DataFrame({'ImageID':image_ids,'LabelName':labels,'Conf':confs})
    df[['XMin','YMin','XMax','YMax']] = boxes
    df['LabelName'] = df['LabelName'].map({0:'opacity',1:'none'})
    df = df[['ImageID','LabelName','Conf','XMin','XMax','YMin','YMax']]    
    return df

In [ ]:
df_test_opacity = combine_boxes(df_test_opacity)
df_test_opacity.shape

In [ ]:
df_test_opacity = df_test_opacity.merge(test_df[['id','width','height']],how='left',left_on='ImageID',right_on='id')
df_test_opacity.head()

In [ ]:
mask = df_test_opacity['LabelName']=='opacity'

w = df_test_opacity.loc[mask,'width'].values
x1x2 = df_test_opacity.loc[mask,['XMin','XMax']].values
x1x2 = x1x2 * w[:,None]
df_test_opacity.loc[mask,['XMin','XMax']] = x1x2

h = df_test_opacity.loc[mask,'height'].values
y1y2 = df_test_opacity.loc[mask,['YMin','YMax']].values
y1y2 = y1y2 * h[:,None]
df_test_opacity.loc[mask,['YMin','YMax']] = y1y2

df_test_opacity.loc[~mask,['XMin','XMax','YMin','YMax']] = np.array([0,1,0,1])

df_test_opacity.shape, mask.sum()

In [ ]:
df_test_opacity=df_test_opacity[dfs_test_none2.columns].copy()

In [ ]:
df_image_pred_test = pd.concat([df_test_opacity,dfs_test_none2])
df_image_pred_test

In [ ]:
SAMPLE_SUBMISSION

In [ ]:
sub_image_ids = np.array([item for item in SAMPLE_SUBMISSION.index if item.endswith('_image')])
sub_image_ids.shape

In [ ]:
def create_str(id_):
    df_tmp = df_image_pred_test[df_image_pred_test['ImageID'] == id_].copy()
    preds = df_tmp[['LabelName','Conf','XMin','YMin','XMax','YMax']].values
    preds = preds.reshape(-1)
    return ' '.join([str(i) for i in preds])

In [ ]:
strs = [create_str(id_) for id_ in sub_image_ids]

In [ ]:
SAMPLE_SUBMISSION.loc[sub_image_ids,'PredictionString'] = strs

In [ ]:
SAMPLE_SUBMISSION.to_csv('submission_image_ch_blend3.csv')

In [ ]:
SAMPLE_SUBMISSION

In [ ]:
SAMPLE_SUBMISSION[SAMPLE_SUBMISSION.PredictionString!=""]

In [ ]:
SAMPLE_SUBMISSION.to_csv('submission.csv')

In [ ]:
!rm -r test_025

In [ ]:
!rm -r test_05

In [ ]:
!rm -rd run_ch_1b

In [ ]:
!rm -rd run_ch_2b

In [ ]:
!rm -rd run_ch_3b

In [ ]:
!rm -rd run_ch_7_2cls_2

In [ ]:
!rm -rd configs

In [ ]:
!rm -rd models